In [14]:
import MetaTrader5 as mt5
from backtesting import Backtest, Strategy
import pandas as pd
from datetime import datetime
import pytz

# Function to calculate SL and TP prices
def calculate_prices(entry_price, risk_reward_ratio, order_type, mean_candle_size):
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    
    if order_type == 'buy':
        sl_price = entry_price - risk_amount
        tp_price = entry_price + reward_amount
    elif order_type == 'sell':
        sl_price = entry_price + risk_amount
        tp_price = entry_price - reward_amount
    else:
        raise ValueError("order_type must be either 'buy' or 'sell'")
    
    return sl_price, tp_price

# Set timezone to UTC
utc_tz = pytz.utc

# Initialize MT5 connection
mt5.initialize()
login = 51708234
password = "4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
if not mt5.login(login, password, server):
    print("Failed to login to MT5")
    mt5.shutdown()
    exit()

symbol = "USDCAD"
timeframe = mt5.TIMEFRAME_D1

# Define start date and end date
start_date = datetime(2022, 3, 10, 0, 0, 0, tzinfo=utc_tz)
end_date = datetime(2024, 3, 1, 0, 0, 0, tzinfo=utc_tz)

# Retrieve OHLC data from MetaTrader
ohlc_data = pd.DataFrame(mt5.copy_rates_range(symbol, timeframe, start_date, end_date))
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
ohlc_data.set_index('time', inplace=True)

# Select only the required columns
df = ohlc_data[['open', 'high', 'low', 'close']]
df.columns = ['Open', 'High', 'Low', 'Close'] 

# Load predictions from CSV file
df_pred = pd.read_csv('prediction2000buy90.csv', index_col=0)
df_pred.index = pd.to_datetime(df_pred.index)

class SimpleStrategy(Strategy):
    mean_candle_size = 0.0074
    risk_reward_ratio = '1:3'  # Default value, will be overridden during optimization

    def init(self):
        self.risk_reward_ratio = self.risk_reward_ratio  # Set risk_reward_ratio from parameters

    def next(self):
        current_time = self.data.index[-1]
        if current_time in df_pred.index:
            prediction = df_pred.loc[current_time, 'prediction']
            entry_price = self.data.Close[-1]
            
            if prediction == 1:
                order_type = 'buy'
                sl_price, tp_price = calculate_prices(entry_price, self.risk_reward_ratio, order_type, self.mean_candle_size)
                self.buy(size=100000, sl=sl_price, tp=tp_price)  # Buy if prediction is 1

# List of risk-reward ratios to test
risk_reward_ratios = ['1:1', '1:2', '1:3', '1:4']

# Store results for each risk-reward ratio
results = []

# Run backtest for each risk-reward ratio
for ratio in risk_reward_ratios:
    print(f"Testing risk-reward ratio: {ratio}")
    bt = Backtest(df, SimpleStrategy, cash=10000, commission=0.0003, margin=0.01)
    stats = bt.run(risk_reward_ratio=ratio)
    results.append((ratio, stats))
    print(stats)  # Print the results for the current risk-reward ratio

# Shutdown MT5 connection after data retrieval
mt5.shutdown()

# Print all results
for ratio, stats in results:
    print(f"\nResults for risk-reward ratio {ratio}:")
    print(stats)

# Optionally plot the last backtest results
bt.plot()


Testing risk-reward ratio: 1:1
Start                     2022-03-10 00:00:00
End                       2024-03-01 00:00:00
Duration                    722 days 00:00:00
Exposure Time [%]                   14.980545
Equity Final [$]                   22557.6453
Equity Peak [$]                    24510.0976
Return [%]                         125.576453
Buy & Hold Return [%]                6.194198
Return (Ann.) [%]                   49.008201
Volatility (Ann.) [%]                43.85733
Sharpe Ratio                         1.117446
Sortino Ratio                        3.353488
Calmar Ratio                         2.863574
Max. Drawdown [%]                  -17.114345
Avg. Drawdown [%]                   -5.839117
Max. Drawdown Duration      139 days 00:00:00
Avg. Drawdown Duration       25 days 00:00:00
# Trades                                   47
Win Rate [%]                        68.085106
Best Trade [%]                       0.623157
Worst Trade [%]                     -0.636864
Avg

c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p8135', ...)

In [8]:
import pandas as pd
from datetime import datetime
import pytz
from backtesting import Backtest, Strategy
import MetaTrader5 as mt5

# Function to calculate SL and TP prices
def calculate_prices(entry_price, risk_reward_ratio, order_type, mean_candle_size):
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    
    if order_type == 'buy':
        sl_price = entry_price - risk_amount
        tp_price = entry_price + reward_amount
    else:
        raise ValueError("order_type must be 'buy' for this strategy")

    return sl_price, tp_price

# Set timezone to UTC
utc_tz = pytz.utc

# Initialize MT5 connection
if not mt5.initialize():
    print("Failed to initialize MT5")
    exit()

login = 51708234
password = "4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
if not mt5.login(login, password, server):
    print("Failed to login to MT5")
    mt5.shutdown()
    exit()

symbol = "USDCAD"
timeframe = mt5.TIMEFRAME_D1

# Define start date and end date
start_date = datetime(2022, 1, 25, 0, 0, 0, tzinfo=utc_tz)
end_date = datetime(2024, 6, 13, 0, 0, 0, tzinfo=utc_tz)

# Retrieve OHLC data from MetaTrader
rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
if rates is None:
    print("Failed to retrieve data")
    mt5.shutdown()
    exit()

ohlc_data = pd.DataFrame(rates)
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
ohlc_data.set_index('time', inplace=True)

# Select only the required columns
df = ohlc_data[['open', 'high', 'low', 'close']]
df.columns = ['Open', 'High', 'Low', 'Close']

# Load predictions for D1 buy signals from CSV file
df_predbuy = pd.read_csv('predictadaboostUSDCAD_D1Buy.csv', index_col=0)
df_predbuy.index = pd.to_datetime(df_predbuy.index)

# Define a simple strategy based on predictions for D1 buy signals
class SimpleStrategy(Strategy):
    mean_candle_size = 0.0090  # Adjusted mean candle size for D1 timeframe

    def init(self):
        pass

    def next(self):
        current_time = self.data.index[-1]
        
        # Get prediction for the current time
        buy_prediction = df_predbuy.loc[current_time, 'prediction'] if current_time in df_predbuy.index else 0
        
        # Place a buy order if the buy prediction is 1
        if buy_prediction == 1:
            entry_price = self.data.Close[-1]
            order_type = 'buy'
            sl_price, tp_price = calculate_prices(entry_price, '2:4', order_type, self.mean_candle_size)  # Adjusted risk-reward ratio
            self.buy(size=100000, sl=sl_price, tp=tp_price)

# List of risk-reward ratios to test for buy orders only
risk_reward_ratios = ['1:1', '1:2', '1:3', '1:4', '2:2', '2:3', '2:4', '2:5']

# DataFrame to store results for each risk-reward ratio
results_columns = ['Equity Final [$]', 'Equity Peak [$]', 'Return [%]', 'Max. Drawdown [%]',
                   'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration', 'Win Rate [%]']
results_df = pd.DataFrame(index=risk_reward_ratios, columns=results_columns)

# Run backtest for each risk-reward ratio
for buy_ratio in risk_reward_ratios:
    print(f"Testing risk-reward ratio: buy={buy_ratio}")
    bt = Backtest(df, SimpleStrategy, cash=10000, commission=0.0003, margin=0.01)
    stats = bt.run()
    
    # Store the results in the DataFrame
    results_df.loc[buy_ratio, 'Equity Final [$]'] = stats['Equity Final [$]']
    results_df.loc[buy_ratio, 'Equity Peak [$]'] = stats['Equity Peak [$]']
    results_df.loc[buy_ratio, 'Return [%]'] = stats['Return [%]']
    results_df.loc[buy_ratio, 'Max. Drawdown [%]'] = stats['Max. Drawdown [%]']
    results_df.loc[buy_ratio, 'Avg. Drawdown [%]'] = stats['Avg. Drawdown [%]']
    results_df.loc[buy_ratio, 'Max. Drawdown Duration'] = stats['Max. Drawdown Duration']
    results_df.loc[buy_ratio, 'Avg. Drawdown Duration'] = stats['Avg. Drawdown Duration']
    results_df.loc[buy_ratio, 'Win Rate [%]'] = stats['Win Rate [%]']

# Shutdown MT5 connection after data retrieval
mt5.shutdown()

# Print the results matrix
print("\nResults matrix:")
print(results_df)

# Optionally plot the last backtest results
bt.plot()


Testing risk-reward ratio: buy=1:1
Testing risk-reward ratio: buy=1:2
Testing risk-reward ratio: buy=1:3
Testing risk-reward ratio: buy=1:4
Testing risk-reward ratio: buy=2:2
Testing risk-reward ratio: buy=2:3
Testing risk-reward ratio: buy=2:4
Testing risk-reward ratio: buy=2:5

Results matrix:
    Equity Final [$] Equity Peak [$] Return [%] Max. Drawdown [%]  \
1:1              0.0       21229.421     -100.0            -100.0   
1:2              0.0       21229.421     -100.0            -100.0   
1:3              0.0       21229.421     -100.0            -100.0   
1:4              0.0       21229.421     -100.0            -100.0   
2:2              0.0       21229.421     -100.0            -100.0   
2:3              0.0       21229.421     -100.0            -100.0   
2:4              0.0       21229.421     -100.0            -100.0   
2:5              0.0       21229.421     -100.0            -100.0   

    Avg. Drawdown [%] Max. Drawdown Duration Avg. Drawdown Duration  \
1:1       

c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p3089', ...)

In [15]:
results_df_sorted = results_df.sort_values(by='Max. Drawdown Duration', ascending=True)

In [16]:
results_df_sorted

,Equity Final [$],Equity Peak [$],Return [%],Max. Drawdown [%],Avg. Drawdown [%],Max. Drawdown Duration,Avg. Drawdown Duration,Win Rate [%]
1:1,0.0,21229.421,-100.0,-100.0,-35.674133,828 days 00:00:00,145 days 00:00:00,0.0
1:2,0.0,21229.421,-100.0,-100.0,-35.674133,828 days 00:00:00,145 days 00:00:00,0.0
1:3,0.0,21229.421,-100.0,-100.0,-35.674133,828 days 00:00:00,145 days 00:00:00,0.0
1:4,0.0,21229.421,-100.0,-100.0,-35.674133,828 days 00:00:00,145 days 00:00:00,0.0
2:2,0.0,21229.421,-100.0,-100.0,-35.674133,828 days 00:00:00,145 days 00:00:00,0.0
2:3,0.0,21229.421,-100.0,-100.0,-35.674133,828 days 00:00:00,145 days 00:00:00,0.0
2:4,0.0,21229.421,-100.0,-100.0,-35.674133,828 days 00:00:00,145 days 00:00:00,0.0
2:5,0.0,21229.421,-100.0,-100.0,-35.674133,828 days 00:00:00,145 days 00:00:00,0.0
